In [1]:
# Phase 1: Environment Setup
!pip install transformers spacy yfinance newsapi-python
!python -m spacy download en_core_web_trf

print("Environment Ready!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 1.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Environment Ready!


In [7]:
import pandas as pd
from newsapi import NewsApiClient
import yfinance as yf

In [3]:
# 1. Initialize API (Replace 'YOUR_API_KEY' with your actual key)
newsapi = NewsApiClient(api_key='e5627542cb9e4af793e97a828ec134aa')

def get_company_news(query):
    # Fetch headlines about the company from the last 30 days
    articles = newsapi.get_everything(q=query,
                                      language='en',
                                      sort_by='relevancy',
                                      page_size=20)

    # Extract only what we need: Date, Title, and Source
    news_data = []
    for art in articles['articles']:
        news_data.append({
            'date': art['publishedAt'],
            'title': art['title'],
            'source': art['source']['name'],
            'description': art['description']
        })

    return pd.DataFrame(news_data)

In [4]:
# Test it
df_apple = get_company_news('Apple Inc')
df_apple.head()

,date,title,source,description
0,2026-01-28T12:37:42Z,Snap is turning its smart glasses team into it...,The Verge,Snap is launching a separate business for its ...
1,2026-01-28T17:19:00Z,Snap Is Spinning Out Its Specs AR Glasses Busi...,CNET,"The subsidiary, Specs Inc., will work on produ..."
2,2026-02-07T17:34:00Z,Apple Plans to Allow Outside Voice-Controlled ...,Slashdot.org,"Apple ""is preparing to allow voice-controlled ..."
3,2026-01-26T13:31:41Z,"149M logins exposed in unsecured database, inc...",9to5Mac,A database containing 149 million account logi...
4,2026-02-09T19:10:50Z,Apple customers receive payments from $95M set...,Newsnationnow.com,The lawsuit alleged Siri recorded private conv...


In [8]:
# Checking without key using yFinance
def get_ticker_news(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    news_list = ticker.news

    # Check if news exists
    if not news_list:
        print(f"No news found for {ticker_symbol}")
        return pd.DataFrame()

    # Convert to DataFrame
    df = pd.DataFrame(news_list)

    # 2026 Fix: Print columns to see what Yahoo is currently sending
    print(f"Current Columns: {df.columns.tolist()}")

    # Select columns only if they exist, otherwise use 'title' as a fallback
    available_cols = [col for col in ['link', 'publisher', 'title'] if col in df.columns]

    if not available_cols:
        # If the expected columns are missing, just return the whole thing
        # so you can see what's inside
        return df

    return df[available_cols]

In [9]:
# Test it
df_tesla = get_ticker_news('TSLA')
print(df_tesla.head())

Current Columns: ['id', 'content']
                                     id  \
0  473e10df-35bc-34e9-b1a1-a088ff4f046f   
1  54842dae-3ced-33bd-b3f4-3f4a1978762e   
2  af1600bd-5924-3e2b-9cfa-d6a9a5d559ed   
3  f9255385-f908-322b-9ce8-aecdbffb80f5   
4  c42b20fe-cd5e-33f0-81ed-83be33561e27   

                                             content  
0  {'id': '473e10df-35bc-34e9-b1a1-a088ff4f046f',...  
1  {'id': '54842dae-3ced-33bd-b3f4-3f4a1978762e',...  
2  {'id': 'af1600bd-5924-3e2b-9cfa-d6a9a5d559ed',...  
3  {'id': 'f9255385-f908-322b-9ce8-aecdbffb80f5',...  
4  {'id': 'c42b20fe-cd5e-33f0-81ed-83be33561e27',...  


In [10]:
df_apple.to_csv('raw_news_data.csv', index=False)
print("Data saved successfully!")

Data saved successfully!
